In [ ]:
!pip install surprise
!pip install gradio
!pip install BeautifulSoup4
!pip install requests

import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
import gradio as gr
import requests
from bs4 import BeautifulSoup

import numpy as np
import random
random.seed(23)
np.random.seed(23)

import logging
logger = logging.getLogger(__name__)

logging.basicConfig(filename='myapp.log', level=logging.CRITICAL)
logger.info('Started')

#Abrimos el recién descargado
file_path = 'dataset_goodreads_with_sentiment.csv'
df = pd.read_csv(file_path)

#Miramos las primeras filas
print(df.head())

#Filtrar usuarios y libros con pocos ratings
min_ratings_user = 5  # Ajusta el umbral según necesidad
min_ratings_book = 5

# Filtrar libros con al menos min_ratings_book ratings
filtered_books = df.groupby('book_id').filter(lambda x: len(x) >= min_ratings_book)

# Filtrar usuarios con al menos min_ratings_user ratings
filtered_df = filtered_books.groupby('user_id').filter(lambda x: len(x) >= min_ratings_user)

# Paso 3: Definir el lector para surprise (formato de los datos)
reader = Reader(rating_scale=(1, 5))  # Define la escala de calificaciones de 1 a 5

# Paso 4: Convertir los datos de pandas a surprise dataset
data = Dataset.load_from_df(filtered_df[['user_id', 'book_id', 'rating']], reader)

# Paso 5: Dividir los datos en conjunto de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2, random_state=23)

# Paso 6: Crear el modelo KNN utilizando la métrica cosine
sim_options = {'name': 'pearson', 'user_based': False}  # 'user_based': True si quieres similitud entre usuarios
knn = KNNBasic(k=20, sim_options=sim_options)

# Paso 7: Entrenar el modelo
knn.fit(trainset)

# Paso 8: Predecir en el conjunto de prueba
predictions = knn.test(testset)

# Paso 9: Evaluar la precisión del modelo
accuracy.rmse(predictions)

# Mapeo de los nombres a los user_id en el dataset
name_user_id = {
    'Maria': '000a1016fda6008d1edbba720ca00851',  # ID correspondiente a Maria
    'Peter': 'ba68089466d16e33ff2f18d375511b17',  # ID correspondiente a Peter
    'Adam': '0011e1a9112b3d798702ef5b20bbf35b',   # ID correspondiente a Adam
    'Andrea': '000efb30c5236d7437c3cdf4bf3e4dc7',  # ID correspondiente a Andrea
    'Lucrecia': '0019de4561419b7543238e0979f2f33e' # ID correspondiente a Lucrecia
}  

def resolve_title_author(book_id):
    # URL de la página que quieres hacer scraping

    url = 'https://www.goodreads.com/book/show/'+str(book_id)  # Reemplaza con la URL real

    # Obtener el contenido de la página web
    response = requests.get(url)
    book_title=""
    author_name=""
    # Verificar que la petición fue exitosa
    if response.status_code == 200:
        # Parsear el contenido HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Buscar el elemento con el atributo data-testid="bookTitle"
        book_title_element = soup.find(attrs={"data-testid": "bookTitle"})
        metadata_section = soup.find(class_='BookPageMetadataSection')

        # Obtener el texto dentro del elemento
        if book_title_element:
            book_title = book_title_element.get_text()
            logger.info(f"El título del libro es: {book_title}")
        else:
            logger.info("No se encontró el elemento con data-testid='bookTitle'")
            
            
        if metadata_section:
        # Buscar el span que tiene 'class="ContributorLink__name"' y 'data-testid="name"'
            contributor_name = metadata_section.find('span', attrs={"class": "ContributorLink__name", "data-testid": "name"})

            # Obtener el texto del elemento
            if contributor_name:
                author_name = contributor_name.get_text()
                logger.info(f"Nombre del autor: {author_name}")
            else:
                logger.info("No se encontró el 'span' con class='ContributorLink__name' y data-testid='name'")
        else:
            logger.info("No se encontró la sección 'BookPageMetadataSection'")
        
    else:
        logger.info(f"Error al acceder a la página, código de estado: {response.status_code}")
        
        
    

    return book_title, author_name

import gradio as gr

def predict_books_for_id(user_id):
   # Obtener todos los libros (ítems) únicos del DataFrame filtrado
    all_books = filtered_df['book_id'].unique()

    # Obtener los libros que el usuario ha calificado en el conjunto de entrenamiento
    user_rated_books = [r[0] for r in trainset.ur[trainset.to_inner_uid(user_id)]]

    # Identificar los libros no leídos
    unread_books = [book for book in all_books if book not in user_rated_books]

    recommended_books = []
    # Predecir la calificación para cada libro no leído
    for book_id in unread_books:
        prediccion = knn.predict(user_id, book_id)
        recommended_books.append((book_id, prediccion.est))

    # Ordenar los libros por el rating predicho en orden descendente
    recommended_books = sorted(recommended_books, key=lambda x: x[1], reverse=True)
    return recommended_books

def recommend_books(name, number_books):
    name_user_id = {
        'Maria': '000a1016fda6008d1edbba720ca00851',
        'Peter': 'ba68089466d16e33ff2f18d375511b17',
        'Adam': '0011e1a9112b3d798702ef5b20bbf35b',
        'Andrea': '000efb30c5236d7437c3cdf4bf3e4dc7',
        'Lucrecia': '0019de4561419b7543238e0979f2f33e'
    }
    
    user_id = name_user_id[name]
    top_n = int(number_books)
    recommended_books = predict_books_for_id(user_id)
    
    final_text = f"Recommended books for {name}:\n"
    for book_id, pred_rating in recommended_books[:top_n]:
        #final_text = final_text + f"Libro ID: {book_id}, Rating Predicho: {pred_rating}\n"
        book_title, author_name = resolve_title_author(book_id)
        final_text = final_text + f"Book: {book_title}, Author: {author_name}, Rating: {pred_rating}\n"
        
    return final_text

# Create a Gradio interface for the function
demo = gr.Interface(fn=recommend_books, inputs=["text", "text"], outputs="text")

# When the user hits 'Flag' Button, your application will persist the value into a file. 

# Launch the interface
demo.launch()